#### FUAM deployment script

##### Before you run this script, please:
1. **Create** a 'FUAM_Config_Lakehouse'
2. **Upload** the 'deployment_file.json' json file to 'deployment' subfolder
3. **Change** the ids of the connections
4. **Run** this notebook

![FUAM deployment process step 3]("https://github.com/GT-Analytics/fuam-basic/blob/main/assets/FUAM_basic_deployment_process_cover_3.png?raw=true")

In [ ]:
%%configure -f

{ 
    "defaultLakehouse": { 
        "name":  "FUAM_Config_Lakehouse"
           }
}

##### Connection IDs (see step 3.)

In [ ]:
# target connections (native)
conn_pbi_service_api_admin = 'abcdef-5a4d-4ad3-bf59-b31f070d72c3'
conn_fabric_service_api_admin = 'ghijkl-8761-4f19-b1b7-a40bfe0ec2af'

##### Deployment logic

In [ ]:
# Helper variables
fuam_lakehouse_datasets = ['FUAM_Basic_PBI_Overview_SM', 'FUAM_Activities_SM', 'FUAM_Capacity_Refreshables_SM']

In [ ]:
import json
import requests
import base64
import time

In [ ]:
# target workspace id
workspace = spark.conf.get("trident.workspace.id")

pbi_access_token = mssparkutils.credentials.getToken("https://analysis.windows.net/powerbi/api")

In [ ]:
# Open deployment json file
deployment = {}
with open(mssparkutils.fs.getMountPath('/default') + "/Files/deployment/deployment_file.json") as f:
    deployment = json.load(f)

In [ ]:
guids_to_replace = [{ "old_id" : deployment["old_workspace"] , "new_id" : workspace}]
guids_to_replace.append({ "old_id" : deployment["connections"]["conn_pbi_service_api_admin_old"] , "new_id" : conn_pbi_service_api_admin})
guids_to_replace.append({ "old_id" : deployment["connections"]["conn_fabric_service_api_admin_old"] , "new_id" : conn_fabric_service_api_admin})


In [ ]:
# Get existing items
# (relevant for FUAM release update)
header = {'Content-Type':'application/json','Authorization': f'Bearer {pbi_access_token}'}
url = 'https://api.fabric.microsoft.com/v1/workspaces/'+ workspace +'/items/'
existing_items = requests.get(url=url, headers=header).json()["value"]

In [ ]:
# Function to get ids from existing items
# (relevant for FUAM release update)
def id_for_existing_items ( name , type):
    for it in existing_items:
        if name == it["displayName"] and type == it["type"]:
            return it["id"]
    return "New Item"

In [ ]:
guids_to_replace

In [ ]:
items_to_deploy = deployment["items"]

In [ ]:
# Function to check if existing items
# (relevant for FUAM release update)
def check_if_item_exists(old_id):
    for row in guids_to_replace:
        if old_id == row['old_id']:
            return True
    return False

In [ ]:
# Deploy items one by one to workspace
# if item new, then create it
# if exists already, then update it
for item in items_to_deploy:
    rename_item = {}
    rename_item["old_id"] = item["org_id"]

    print('Deploy ' + item['displayName'] )  

    if 'definition' in item.keys():
        b = item['definition']['parts'][0]['payload']
        decoded = base64.b64decode(b).decode('utf-8')

        for repl in guids_to_replace:
            decoded = decoded.replace(repl["old_id"], repl["new_id"])
        encoded = base64.b64encode(decoded.encode('utf-8'))
        item['definition']['parts'][0]['payload'] = encoded

    it = item
    header = {'Content-Type':'application/json','Authorization': f'Bearer {pbi_access_token}'}

    existing_id = id_for_existing_items(item['displayName'], item['type'])
    if existing_id == "New Item":
        print( "Create ")
        url = 'https://api.fabric.microsoft.com/v1/workspaces/'+ workspace + '/items/'
        response = requests.post(url=url, headers=header, json = item)
    else:
        print( "Update ")
        url = 'https://api.fabric.microsoft.com/v1/workspaces/'+ workspace + '/items/' + existing_id + "/updateDefinition"
        response = requests.post(url=url, headers=header, json = item) 

    if response.status_code == 202:
        get_op = 'Running'
        while get_op != 'Succeeded' and get_op != 'Failed':
            time.sleep(1.5)
                
            header = {'Content-Type':'application/json','Authorization': f'Bearer {pbi_access_token}'}
            response2 = requests.get(url=response.headers["location"], headers=header)
            get_op = response2.json()['status']
            print(get_op)

            header = {'Content-Type':'application/json','Authorization': f'Bearer {pbi_access_token}'}

            response3 = requests.get(url=response.headers["location"]+ "/result", headers=header)
            response3 = response3.json()
    else:
        if existing_id == "New Item":
            response3 = response.json()
    if existing_id == "New Item":
        rename_item["new_id"] = response3["id"]
    else:
        rename_item["new_id"] = existing_id
    guids_to_replace.append(rename_item)

In [ ]:
# Get existing items after deployment
header = {'Content-Type':'application/json','Authorization': f'Bearer {pbi_access_token}'}
url = 'https://api.fabric.microsoft.com/v1/workspaces/'+ workspace +'/items/'
existing_items = requests.get(url=url, headers=header).json()["value"]

In [ ]:
# Get SQL Endpoint properties for main Lakehouse
header = {'Content-Type':'application/json','Authorization': f'Bearer {pbi_access_token}'}
url = 'https://api.fabric.microsoft.com/v1/workspaces/'+ workspace +'/lakehouses/' + id_for_existing_items('FUAM_Lakehouse', 'Lakehouse')
response = requests.get(url=url, headers=header)
new_sqlEndPointProperties = response.json()['properties']['sqlEndpointProperties']

In [ ]:
new_sqlEndPointProperties

In [ ]:
# Set SQL Endpoint
old_sql_EndPointProperties = deployment['sqlEndPointProperties']
old_sql_EndPointProperties

In [ ]:
update_datasource_json = {}
updateDetails = []
single_updateDetails = {}
single_updateDetails['datasourceSelector'] = {}
single_updateDetails['datasourceSelector']['datasourceType'] = "Sql"
single_updateDetails['datasourceSelector']["connectionDetails"] = {}
single_updateDetails['datasourceSelector']["connectionDetails"]["server"] = old_sql_EndPointProperties['connectionString']
single_updateDetails['datasourceSelector']["connectionDetails"]["database"] = old_sql_EndPointProperties['id']

single_updateDetails['connectionDetails'] = {}
single_updateDetails['connectionDetails']["server"] = new_sqlEndPointProperties['connectionString']
single_updateDetails['connectionDetails']["database"] = new_sqlEndPointProperties['id']

updateDetails.append(single_updateDetails)
update_datasource_json['updateDetails'] = updateDetails

In [ ]:
update_datasource_json

In [ ]:
# Update connection between semantic model and lakehouse
for sm in fuam_lakehouse_datasets:
    print(sm)
    
    header = {'Content-Type':'application/json','Authorization': f'Bearer {pbi_access_token}'}
    url = 'https://api.powerbi.com/v1.0/myorg/datasets/'+  id_for_existing_items(sm, 'SemanticModel') + '/Default.UpdateDatasources' 
    response = requests.post(url=url, headers=header, json = update_datasource_json)
    print(response.status_code)
